In [1]:
import pandas as pd
import numpy as np
from surprise import Dataset, Reader
from surprise.model_selection import train_test_split
from surprise import SVD
from surprise import accuracy

# Завантаження даних з файлів для рекомендаційної системи
tags_df = pd.read_csv('C:\\Users\\Сергій\\Documents\\Code\\Current_task\\Math-test\\task7\\tags.csv')
movies_df = pd.read_csv('C:\\Users\\Сергій\\Documents\\Code\\Current_task\\Math-test\\task7\\movies.csv')
links_df = pd.read_csv('C:\\Users\\Сергій\\Documents\\Code\\Current_task\\Math-test\\task7\\links.csv')
ratings_df = pd.read_csv('C:\\Users\\Сергій\\Documents\\Code\\Current_task\\Math-test\\task7\\ratings_filtered.csv')  # Використовуйте ratings_filtered.csv

# З'єднуємо файли за movieId
merged_df = movies_df.merge(links_df, on='movieId', how='inner')
merged_df = merged_df.merge(tags_df, on='movieId', how='inner')

# Об'єднуємо з файлом ratings_filtered.csv за userId і movieId
merged_df = merged_df.merge(ratings_df[['userId', 'movieId', 'rating']], on=['userId', 'movieId'], how='inner')

# Визначаємо Reader для бібліотеки surprise
reader = Reader(rating_scale=(0, 5))

# Створюємо датасет з даних
data = Dataset.load_from_df(merged_df[['userId', 'movieId', 'rating']], reader)

# Розділяємо датасет на навчальний та тестовий для рекомендаційної системи
trainset, testset = train_test_split(data, test_size=0.2, random_state=42)

# Побудова моделі SVD та її навчання для рекомендаційної системи
model = SVD()
model.fit(trainset)

# Прогнозування рейтингів для тестового набору для рекомендаційної системи
predictions = model.test(testset)

# Обчислення середньоквадратичної помилки (RMSE) для рекомендаційної системи
rmse = accuracy.rmse(predictions)
print(f'RMSE for Recommender System: {rmse}')

# Завантаження даних про рейтинги користувачів і товарів для матричної факторизації
ratings_matrix = np.array([
    [5, 4, 0, 0, 1],
    [0, 0, 5, 4, 2],
    [3, 0, 0, 0, 5],
    [0, 2, 4, 0, 0],
    [1, 0, 0, 3, 4]
])

# Кількість користувачів і товарів
num_users, num_items = ratings_matrix.shape

# Ініціалізація матриць користувачів і товарів з випадковими значеннями
num_factors = 3  # Кількість факторів
user_matrix = np.random.rand(num_users, num_factors)
item_matrix = np.random.rand(num_items, num_factors)

# Градієнтний спуск для оптимізації функції втрат для матричної факторизації
learning_rate = 0.01
num_iterations = 100

for _ in range(num_iterations):
    # Робимо прогноз рейтингів для матричної факторизації
    predicted_ratings = np.dot(user_matrix, item_matrix.T)

    # Обчислюємо функцію втрат для матричної факторизації
    loss = np.sum((ratings_matrix - predicted_ratings) ** 2)

    # Обчислюємо градієнти для матричної факторизації
    user_gradient = -2 * np.dot((ratings_matrix - predicted_ratings), item_matrix)
    item_gradient = -2 * np.dot((ratings_matrix - predicted_ratings).T, user_matrix)

    # Оновлюємо матриці користувачів і товарів для матричної факторизації
    user_matrix -= learning_rate * user_gradient
    item_matrix -= learning_rate * item_gradient

# Прогнозування рейтингів для нових користувачів і товарів для матричної факторизації
new_user_ratings = np.dot(user_matrix[-1, :], item_matrix.T)
print(f'Predicted Ratings for New User (Matrix Factorization): {new_user_ratings}')


RMSE: 0.4054
RMSE for Recommender System: 0.40541718269429633
Predicted Ratings for New User (Matrix Factorization): [ 1.28842784 -0.60036092  0.53618536  2.20464944  4.16831474]
